In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [4]:
# tool create
@tool
def multiply(a: int, b: int) -> int:
  """Given 2 numbers a and b this tool returns their product"""
  return a * b

In [8]:
print(multiply.invoke({'a':3, 'b':4}))
print(multiply.name)
print(multiply.description)
print(multiply.args)

12
multiply
Given 2 numbers a and b this tool returns their product
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [9]:
llm = HuggingFaceEndpoint(
    model="meta-llama/Llama-3.3-70B-Instruct",
    task="text-generation"
)

model = ChatHuggingFace(llm=llm)

d:\anaconda3\envs\genai_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
model.invoke('hi')

AIMessage(content='Hello. How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 36, 'total_tokens': 46}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': 'fp_c5ec625e72d41732d8fd', 'finish_reason': 'stop', 'logprobs': None}, id='run--124d8dbe-7ae5-42af-b216-eea3c9a742ed-0', usage_metadata={'input_tokens': 36, 'output_tokens': 10, 'total_tokens': 46})

In [ ]:
# tool binding into LLM
llm_with_tools = model.bind_tools([multiply])
llm_with_tools.invoke('Hi how are you')

AIMessage(content="I'm just a language model, I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 246, 'total_tokens': 295}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': 'fp_c5ec625e72d41732d8fd', 'finish_reason': 'stop', 'logprobs': None}, id='run--a62a3777-833b-410e-9509-28d6d92c03bc-0', usage_metadata={'input_tokens': 246, 'output_tokens': 49, 'total_tokens': 295})

In [12]:
query = HumanMessage('can you multiply 3 with 1000')
messages = [query]
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={})]

In [13]:
result = llm_with_tools.invoke(messages)
messages.append(result)
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 3, "b": 1000}', 'name': 'multiply', 'description': None}, 'id': 'd29cfed5e', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 251, 'total_tokens': 266}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': 'fp_c5ec625e72d41732d8fd', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7a062d6f-37fa-4f61-bbdb-67385ae5516b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'd29cfed5e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 251, 'output_tokens': 15, 'total_tokens': 266})]

In [15]:
tool_result = multiply.invoke(result.tool_calls[0])
tool_result

ToolMessage(content='3000', name='multiply', tool_call_id='d29cfed5e')

In [16]:
messages.append(tool_result)
messages

[HumanMessage(content='can you multiply 3 with 1000', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 3, "b": 1000}', 'name': 'multiply', 'description': None}, 'id': 'd29cfed5e', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 251, 'total_tokens': 266}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': 'fp_c5ec625e72d41732d8fd', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--7a062d6f-37fa-4f61-bbdb-67385ae5516b-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': 'd29cfed5e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 251, 'output_tokens': 15, 'total_tokens': 266}),
 ToolMessage(content='3000', name='multiply', tool_call_id='d29cfed5e')]

In [19]:
llm_with_tools.invoke(messages)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"a": 3, "b": 1000}', 'name': 'multiply', 'description': None}, 'id': '4a0ef5715', 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 267, 'total_tokens': 282}, 'model_name': 'meta-llama/Llama-3.3-70B-Instruct', 'system_fingerprint': 'fp_c5ec625e72d41732d8fd', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--cefd489d-8ab1-4896-8a53-3484aea7406c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 1000}, 'id': '4a0ef5715', 'type': 'tool_call'}], usage_metadata={'input_tokens': 267, 'output_tokens': 15, 'total_tokens': 282})

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [ ]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [ ]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1745798401,
 'time_last_update_utc': 'Mon, 28 Apr 2025 00:00:01 +0000',
 'time_next_update_unix': 1745884801,
 'time_next_update_utc': 'Tue, 29 Apr 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.4741}

In [ ]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.16})

851.5999999999999

In [ ]:
# tool binding
llm = ChatOpenAI()

In [ ]:
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
messages = [HumanMessage('What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd')]

In [ ]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={})]

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
messages.append(ai_message)

In [ ]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': 'call_PKL8v7zwmphzNel0MnvjjGvY',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10},
  'id': 'call_vRdld30yHTKFGcTQPumgzH5u',
  'type': 'tool_call'}]

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)



In [ ]:
messages

[HumanMessage(content='What is the conversion factor between INR and USD, and based on that can you convert 10 inr to usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PKL8v7zwmphzNel0MnvjjGvY', 'function': {'arguments': '{"base_currency": "INR", "target_currency": "USD"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_vRdld30yHTKFGcTQPumgzH5u', 'function': {'arguments': '{"base_currency_value": 10}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 123, 'total_tokens': 176, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BR8rovupYuNnIbGO41TOO3knQ2Cr9', 'finish_reason': 'too

In [ ]:
llm_with_tools.invoke(messages).content

'The conversion factor between INR and USD is 0.0117. \n\nBased on this conversion factor, 10 INR is equal to 0.117 USD.'

In [ ]:
from langchain.agents import initialize_agent, AgentType

# Step 5: Initialize the Agent ---
agent_executor = initialize_agent(
    tools=[get_conversion_factor, convert],
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,  # using ReAct pattern
    verbose=True  # shows internal thinking
)

In [ ]:
# --- Step 6: Run the Agent ---
user_query = "Hi how are you?"

response = agent_executor.invoke({"input": user_query})



> Entering new AgentExecutor chain...
I'm here and ready to help! How can I assist you today?

> Finished chain.
